# Comparing the Quality and Loess between a Full and a Half Tesla Cavity Using the EPR approach
Calculation of resonant cavity quality factors, losses, life-time and more using the EPR method. This code is based on [Ansys HFSS](https://www.ansys.com/products/electronics/ansys-hfss) and the [pyEPR library](https://github.com/zlatko-minev/pyEPR).

The cavity is half of a [Tesla cavity](https://arxiv.org/pdf/physics/0003011.pdf), this will be usefule in a later notebook where we compare the full Tesla cavity to our (half Tesla) cavity.

#### Imports

In [1]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy = True

In [2]:
import sys
import numpy as np
from IPython.display import display, Math, Latex, display_markdown
from pathlib import Path
import pandas as pd
from scipy import constants

import pyEPR as epr
from pyEPR.calcs import Convert
from pyEPR.core import *
from pyEPR.ansys import *
import warnings
warnings.simplefilter("ignore")

In [3]:
path_to_project = 'D:\\Users\\Daniel\\Cavity-Analysis'

## 👉 Half Cavity
We start with the interesting one, the half cavity. This is the cavity we're going to be using and we want to check that it isn't that much worse than the full Tesla cavity.

### 🔷 Mode analysis

#### 🔹 Connect to HFSS

In [4]:
pinfo = epr.Project_Info(project_path = path_to_project, 
                         project_name = 'Cavity Analysis',
                         design_name  = 'half cav')

INFO 10:10PM [connect]: Connecting to Ansys Desktop API...
INFO 10:10PM [load_ansys_project]: 	File path to HFSS project found.
INFO 10:10PM [load_ansys_project]: 	Opened Ansys App
INFO 10:10PM [load_ansys_project]: 	Opened Ansys Desktop v2020.1.0
INFO 10:10PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    D:/Users/Daniel/Cavity-Analysis/
	Project:   Cavity Analysis
INFO 10:10PM [connect]: 	Opened active design
	Design:    half cav [Solution type: Eigenmode]
INFO 10:10PM [get_setup]: 	Opened setup `setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 10:10PM [connect]: 	Connection to Ansys established successfully. 😀 



In [5]:
pinfo.setup.analyze()
eprh = epr.DistributedAnalysis(pinfo)

INFO 10:10PM [analyze]: Analyzing setup setup


Design "half cav" info:
	# eigenmodes    1
	# variations    1


#### 🔹 Get HFSS mode and quality results

In [6]:
modes      = eprh.get_freqs_bare_pd(eprh.variations[0])
Fs, Qs     = np.array(modes['Freq. (GHz)']), np.array(modes['Quality Factor'])  # Get freqs and Q-factors
n_modes    = int(pinfo.setup.n_modes)
display(modes)

,Freq. (GHz),Quality Factor
mode,,
0,7.652455,inf


#### 🔹 Calculate the EPRs of the modes

In [7]:
eprh.set_mode(0)
p_cavity, (ℰ_cav, ℰ_total) = eprh.calc_p_electric_volume('cavity')
p_dirt, (ℰ_dirt, ℰ_total) = eprh.calc_p_electric_volume('dirt')

print(f' 🔸 Energy in cavity = {100*p_cavity:.3f}% -> {ℰ_cav:0.2e} of the total energy in the system')
print(f' 🔸 Energy in dirt   = {100*p_dirt:.3f}% -> {ℰ_dirt:0.2e} of the total energy in the system')
print(f' 🔸 Total energy     = {ℰ_total:0.2e}')

 🔸 Energy in cavity = 99.124% -> 6.09e-18 of the total energy in the system
 🔸 Energy in dirt   = 0.876% -> 5.39e-20 of the total energy in the system
 🔸 Total energy     = 6.15e-18


### 🔷 Life-times

**Life-time from HFSS**

Life time of a mode inside the cavity. Since in this exampole the cavity is *perfect*, the life time would be infinite

In [8]:
Fs_Hz  = np.array(Convert.toSI(Fs,'GHz'))  # Mode freqs in Hz
omegas = 2*np.pi*Fs_Hz                     # Freqs to angular freqs
taus   = Qs/omegas                         # Life times

print(f' 🔸 Life-time of mode = {taus[0]*1e3:.3f} ms')  # Should be inf since no resistive boundry and just inside a vacuum

 🔸 Life-time of mode = inf ms


**Life-time from EPR**

Life time calculation with the EPR method. This is highly dependent on the loss tangent of the dirt and cavity.

In [9]:
tan_dirt   = 4e-7                                         # Loss tangent of dirt

tau_epr    = lambda p, tan, omega: 1/(p*tan*omega)        # Easily calculate life time with EPR

tau_cavity = tau_epr(p_dirt, 0, omegas)[0]
tau_dirt   = tau_epr(p_cavity, tan_dirt, omegas)[0]

print(f' 🔸 Cavity life-time = {tau_cavity*1e6:.2f} ns')  # Should be infinite since the cavity is a pefect vacum
print(f' 🔸 Dirt life-time   = {tau_dirt*1e6:.2f} ns')

 🔸 Cavity life-time = inf ns
 🔸 Dirt life-time   = 52.45 ns


### 🔷 Losses

#### 🔹 Surface loss

Calculating the energy precentage near the cavity walls by the surface integral. The total energy of the electromagnetic field at a layer of thickness `dirt_widht` would be approximated as:
$$\text{E}_{\text{cavity, boundry}} \approx \text{dirt_width} \cdot \int_{S_{cavity}} |E|^2$$
$$\text{E}_{\text{cavity, volume}} = \int_{V_{cavity}} |E|^2$$

$$\text{EPR}_{\text{cavity, boundry}} \approx \frac{\text{E}_{\text{cavity, boundry}}}{\text{E}_{\text{cavity, volume}}}$$

First we'll setup a dictionary to store all the data

In [10]:
data = {
    "half":{
        "volume":{
            
        },
        "surface":{
           
        }
    },
    "full":{
        "volume":{
          
        },
        "surface":{
           
        }
    }
}

In [11]:
dirt_width = 0.1e-3
eps        = 1
tan_surf   = 5e-3

eprh.set_mode(0)

# --- Surface integral ---
surf = 'cavity'
calcobject = CalcObject([], eprh.setup)
vecE = calcobject.getQty("E").smooth()
A = vecE.times_eps()
B = vecE.conj()
A = A.dot(B)
A = A.real()
A = A.integrate_surf(name=surf)

E_subs = A.evaluate(lv=eprh._get_lv()) 
E_surf = E_subs*dirt_width*eps

# --- Volume integral ---
E_total = eprh.calc_energy_electric(smooth=True)

p_surf = E_surf/E_total      # EPR of surface 
Q_surf = 1/tan_surf/p_surf   # Q-fact of surface
tau_surf = Q_surf/omegas[0]  #  Life-time of surface

data['half']['surface'] = {
    "EPR": p_surf,
    "Q":   Q_surf,
    "tau":  tau_surf
}

print(f' 🔸 EPR surface       = {100*p_surf:.2f}%')
print(f' 🔸 Q-factor surface  = {Q_surf:.2e}')
print(f' 🔸 Life-time surface = {tau_surf:.2e} seconds \n')


 🔸 EPR surface       = 0.98%
 🔸 Q-factor surface  = 2.05e+04
 🔸 Life-time surface = 4.27e-07 seconds 



#### 🔹 Dirt (volume) loss

In [12]:
# Dirt is simulated as much thicker than it actually is (for computation reason). 
# Beacuase of that we reduce the loss tangent to an 'effective loss tangent' which is loss_tan*thick_factor
p_dirt, (ℰ_dirt, ℰ_total) = eprh.calc_p_electric_volume('dirt')
Q_dirt = 1/(tan_surf*p_dirt)
tau_dirt = Q_dirt/omegas[0]

data['half']['volume'] = {
    "EPR": p_dirt,
    "Q":   Q_dirt,
    "tau":  tau_dirt
}

print(f'  🔸 EPR of dirt    = {100*p_dirt:0.2f}% ( {ℰ_dirt:.2e} / {ℰ_total:.2e} )')
print(f'  🔸 Quality factor = {Q_dirt:0.2e}')
print(f'  🔸 life time      = {tau_dirt:0.2e} seconds\n')

  🔸 EPR of dirt    = 0.88% ( 5.39e-20 / 6.15e-18 )
  🔸 Quality factor = 2.28e+04
  🔸 life time      = 4.75e-07 seconds



## 👉 Full Cavity

### 🔷 Mode analysis

#### 🔹 Connect to HFSS

In [13]:
pinfo_full = epr.Project_Info(project_path = path_to_project, 
                             project_name = 'Cavity Analysis',
                             design_name  = 'full cav')

INFO 10:10PM [connect]: Connecting to Ansys Desktop API...
INFO 10:10PM [load_ansys_project]: 	File path to HFSS project found.
INFO 10:10PM [load_ansys_project]: 	Opened Ansys App
INFO 10:10PM [load_ansys_project]: 	Opened Ansys Desktop v2020.1.0
INFO 10:10PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    D:/Users/Daniel/Cavity-Analysis/
	Project:   Cavity Analysis
INFO 10:10PM [connect]: 	Opened active design
	Design:    full cav [Solution type: Eigenmode]
INFO 10:10PM [get_setup]: 	Opened setup `setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 10:10PM [connect]: 	Connection to Ansys established successfully. 😀 



In [14]:
pinfo_full.setup.analyze()
eprh_full = epr.DistributedAnalysis(pinfo_full)

INFO 10:10PM [analyze]: Analyzing setup setup


Design "full cav" info:
	# eigenmodes    1
	# variations    1


#### 🔹 Get HFSS mode and quality results

In [15]:
modes      = eprh_full.get_freqs_bare_pd(eprh_full.variations[0])
Fs, Qs     = np.array(modes['Freq. (GHz)']), np.array(modes['Quality Factor'])  # Get freqs and Q-factors
n_modes    = int(pinfo_full.setup.n_modes)
display(modes)

,Freq. (GHz),Quality Factor
mode,,
0,6.518766,inf


#### 🔹 Calculate the EPRs of the modes

In [16]:
eprh_full.set_mode(0)
p_cavity, (ℰ_cav, ℰ_total) = eprh_full.calc_p_electric_volume('cavity')
p_dirt, (ℰ_dirt, ℰ_total) = eprh_full.calc_p_electric_volume('dirt')

print(f' 🔸 Energy in cavity = {100*p_cavity:.3f}% -> {ℰ_cav:0.2e}')
print(f' 🔸 Energy in dirt   = {100*p_dirt:.3f}% -> {ℰ_dirt:0.2e}')
print(f' 🔸 Total energy     = {ℰ_total:0.2e}')

 🔸 Energy in cavity = 99.840% -> 1.24e-17
 🔸 Energy in dirt   = 0.160% -> 1.99e-20
 🔸 Total energy     = 1.24e-17


### 🔷 Life-times

**Life-time from HFSS**

In [17]:
Fs_Hz  = np.array(Convert.toSI(Fs,'GHz'))  # Mode freqs in Hz
omegas = 2*np.pi*Fs_Hz                     # Freqs to angular freqs
taus   = Qs/omegas                         # Life times

print(f' 🔸 Life-time of mode = {taus[0]*1e3:.3f} ms')

 🔸 Life-time of mode = inf ms


**Life-time from EPR**

In [18]:
tan_dirt = 4e-7  # Loss tangent of dirt

tau_epr = lambda p, tan, omega: 1/(p*tan*omega)  # Easily calculate life time with EPR

tau_cavity = tau_epr(p_dirt, 0, omegas)[0]
tau_dirt = tau_epr(p_cavity, tan_dirt, omegas)[0]

print(f' 🔸 Cavity life-time = {tau_cavity*1e6:.2f} ns')  # Should be infinite since the cavity is a pefect vacum
print(f' 🔸 Dirt life-time   = {tau_dirt*1e6:.2f} ns')

 🔸 Cavity life-time = inf ns
 🔸 Dirt life-time   = 61.14 ns


### 🔷 Losses

#### 🔹 Surface loss

In [19]:
dirt_width = 0.1e-3
eps        = 1
tan_surf   = 5e-3

eprh_full.set_mode(0)

# --- Surface integral ---
surf = 'cavity'
calcobject = CalcObject([], eprh_full.setup)
vecE = calcobject.getQty("E").smooth()
A = vecE.times_eps()
B = vecE.conj()
A = A.dot(B)
A = A.real()
A = A.integrate_surf(name=surf)

E_subs = A.evaluate(lv=eprh_full._get_lv()) 
E_surf = E_subs*dirt_width*eps

# --- Volume integral ---
E_total = eprh_full.calc_energy_electric(smooth=True)

p_surf = E_surf/E_total      # EPR of surface 
Q_surf = 1/tan_surf/p_surf   # Q-fact of surface
tau_surf = Q_surf/omegas[0]  #  Life-time of surface

data['full']['surface'] = {
    "EPR":  p_surf,
    "Q":    Q_surf,
    "tau":  tau_surf
}

print(f' 🔸 EPR surface       = {100*p_surf:.2f}%')
print(f' 🔸 Q-factor surface  = {Q_surf:.2e}')
print(f' 🔸 Life-time surface = {tau_surf:.2e} seconds \n')

 🔸 EPR surface       = 0.53%
 🔸 Q-factor surface  = 3.78e+04
 🔸 Life-time surface = 9.22e-07 seconds 



#### 🔹 Dirt (volume) loss

In [20]:
# Dirt is simulated as much thicker than it actually is (for computation reason). 
# Beacuase of that we reduce the loss tangent to an 'effective loss tangent' which is loss_tan*thick_factor
p_dirt, (ℰ_dirt, ℰ_total) = eprh_full.calc_p_electric_volume('dirt')
Q_dirt = 1/(tan_surf*p_dirt)
tau_dirt = Q_dirt/omegas[0]

data['full']['volume'] = {
    "EPR":  p_dirt,
    "Q":    Q_dirt,
    "tau":  tau_dirt
}

print(f'  🔸 EPR of dirt    = {100*p_dirt:0.2f}% ( {ℰ_dirt:.2e} / {ℰ_total:.2e} )')
print(f'  🔸 Quality factor = {Q_dirt:0.2e}')
print(f'  🔸 life time      = {tau_dirt:0.2e} seconds\n')

  🔸 EPR of dirt    = 0.16% ( 1.99e-20 / 1.24e-17 )
  🔸 Quality factor = 1.25e+05
  🔸 life time      = 3.05e-06 seconds



## Conclusion 🎉
We'll compare the full cavity to the half cavity now. First with the surface integral calculation, then with the volume calculation and finally we'll compare the two approaches (that should yield roughly the same result). We'll calculate the ratio between the full cavity calculated parameters and half cavity calculated parameters. 

In [21]:
# ═══ Surface integral ═══
EPR_ratio_surf  = data['full']['surface']['EPR']/data['half']['surface']['EPR']
Q_ratio_surf    = data['full']['surface']['Q']/data['half']['surface']['Q']
tau_ratio_surf  = data['full']['surface']['tau']/data['half']['surface']['tau']

# ═══ Volume integral ═══
EPR_ratio_vol   = data['full']['volume']['EPR']/data['half']['volume']['EPR']
Q_ratio_vol     = data['full']['volume']['Q']/data['half']['volume']['Q']
tau_ratio_vol   = data['full']['volume']['tau']/data['half']['volume']['tau']

# ═══ Volume vs Surface ═══
EPR_surf_vol   = EPR_ratio_surf/EPR_ratio_vol
Q_surf_vol     = Q_ratio_surf/Q_ratio_vol
tau_surf_vol   = tau_ratio_surf/tau_ratio_vol

print('═'*10, '🎉 Results 🎉', '═'*10)
print('Ratio full-to-half cavity parameters\n')

print('\t─── Surface integral ───')
print(f'  🔸 EPR:       {EPR_ratio_surf:.3f}')
print(f'  🔸 Q-factor:  {Q_ratio_surf:.3f}')
print(f'  🔸 Life-time: {tau_ratio_surf:.3f}\n')

print('\t─── Volume integral ───')
print(f'  🔸 EPR:       {EPR_ratio_vol:.3f}')
print(f'  🔸 Q-factor:  {Q_ratio_vol:.3f}')
print(f'  🔸 Life-time: {tau_ratio_vol:.3f}\n')

print('\t─── Volume v Surface ───')
print(f'  🔸 EPR:       {EPR_surf_vol:.3f}')
print(f'  🔸 Q-factor:  {Q_surf_vol:.3f}')
print(f'  🔸 Life-time: {tau_surf_vol:.3f}')

══════════ 🎉 Results 🎉 ══════════
Ratio full-to-half cavity parameters

	─── Surface integral ───
  🔸 EPR:       0.543
  🔸 Q-factor:  1.842
  🔸 Life-time: 2.162

	─── Volume integral ───
  🔸 EPR:       0.183
  🔸 Q-factor:  5.473
  🔸 Life-time: 6.424

	─── Volume v Surface ───
  🔸 EPR:       2.972
  🔸 Q-factor:  0.337
  🔸 Life-time: 0.337


In [47]:
print('Ratio surface to volume of EPR')
print(f'  🔸 Half cavity:  {data["half"]["surface"]["EPR"]/data["half"]["volume"]["EPR"]:.3f}')  # <-- Good
print(f'  🔸 Full cavity:  {data["full"]["surface"]["EPR"]/data["full"]["volume"]["EPR"]:.3f}')  # <-- Bad

Ratio surface to volume of EPR
  🔸 Half cavity:  1.113
  🔸 Full cavity:  3.308


In [50]:
print('Raw data:')
print(data)

Raw data:
{'half': {'volume': {'EPR': 0.008760304743812316, 'Q': 22830.25600693474, 'tau': 4.7482120666914146e-07}, 'surface': {'EPR': 0.009750751644802916, 'Q': 20511.23926498514, 'tau': 4.265905461209644e-07}}, 'full': {'volume': {'EPR': 0.0016007415993501732, 'Q': 124942.08939231087, 'tau': 3.0504470257966513e-06}, 'surface': {'EPR': 0.005294800493185226, 'Q': 37772.905751106926, 'tau': 9.222212351705111e-07}}}


In [51]:
pinfo.disconnect()